In [1]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from dotenv import load_dotenv
import os
from pydantic import Field
from deep_research import Deep_research_engine
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dataclasses import dataclass
from presentation_generator import Presentation_gen
from typing import Optional

In [2]:
load_dotenv()
tavily_key=os.getenv('tavily_key')
google_api_key=os.getenv('google_api_key')

llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

deepsearch=Deep_research_engine()
presentation=Presentation_gen()

@dataclass
class Deps:
    deep_search_results:list[str]
    quick_search_results:list[str]
    edited_deep_search_result:str
    presentation_slides:dict



async def deep_research_agent(ctx:RunContext[Deps], query:str):
    """
    This function is used to do a deep research on the web for information on a complex query, generates a report or a paper.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await deepsearch.chat(query)
    ctx.deps.deep_search_results.append(res)
    return str(res)

quick_search_agent=Agent(llm,tools=[tavily_search_tool(tavily_key)])
async def quick_research_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to do a quick search on the web for information on a given query.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await quick_search_agent.run(query)
    ctx.deps.quick_search_results.append(res.data)
    return str(res.data)

@dataclass
class Research_edits:
    edits:str = Field(description='the edited reasearch')
editor_agent=Agent(llm,result_type=Research_edits, system_prompt="you are an editor, you are given a query, a deep search result, and maybe a quick search result (optional), you need to edit the deep search result to make it more accurate following the query's instructions")
async def research_editor_tool(ctx: RunContext[Deps], query:str):
    """
    This function is used to edit the deep search result to make it more accurate following the query's instructions.
    Args:
        query (str): The query containing instructions for editing the deep search result
    Returns:
        str: The edited and improved deep search result
    """
    res=await editor_agent.run(f'query:{query}, deep_search_result:{ctx.deps.deep_search_results[0]}, quick_search_result:{ctx.deps.quick_search_results[0]}')
    ctx.deps.edited_deep_search_result=res.data.edits
    return str(res.data.edits)


async def presentation_agent(
    ctx: RunContext[Deps],
    style: Optional[str] = "professional",
    instruction: Optional[str] = 'None'
) -> dict:
    """
    Generate a presentation based on the deep search result.

    Args:
        style: Presentation style (default: "professional")
        instruction: Additional instructions for presentation generation (default: None)

    Returns:
        dict: The presentation slides
    """
    search_content = (ctx.deps.edited_deep_search_result 
                     if ctx.deps.edited_deep_search_result 
                     else ctx.deps.deep_search_results[0])
    
    res = await presentation.chat(search_content, presentation_style=style, instruction=instruction)
    ctx.deps.presentation_slides = res
    return 'presentation generated'

@dataclass
class page_edit:
    edits: str = Field(description='the edited page in html format')

page_editor=Agent(llm, result_type=page_edit, system_prompt='you are a page editor, edit the mentionned page(s) based on the instructions.')
async def page_editor_tool(ctx: RunContext[Deps], page_number: int, instructions: str):
    """
    This function is used to edit the mentionned page(s) based on the instructions.
    Args:
        page_number (int): The number of the page to edit
        instructions (str): The instructions for the page edits
    Returns:
        str: The edited page
    """
    result=await page_editor.run(f'page:{ctx.deps.presentation_slides[f"page_{page_number}"]}, instructions:{instructions}')
    ctx.deps.presentation_slides[f"page_{page_number}"]=result.data.edits
    return f'page{page_number} edited'


In [3]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

In [4]:
class Main_agent:
    def __init__(self):
        self.agent=Agent(llm, system_prompt="you are a research assistant, you are given a query, leverage what tool(s) to use,\
                          always show the output of the tools, except for the presentation agent",
                          tools=[deep_research_agent,page_editor_tool,presentation_agent,research_editor_tool,quick_research_agent])
        self.deps=Deps( deep_search_results=[], quick_search_results=[],edited_deep_search_result="", presentation_slides={})
        self.memory=Message_state(messages=[])

    async def chat(self, query:str):
        result = await self.agent.run(query,deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.data

In [5]:
agent=Main_agent()

In [6]:
await agent.chat('do a deep research on the blind snake in the bahamas and generate a presentation about it')

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\deep_research_agent\deep_research.py:98: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  response=await self.graph.run(preliminary_search_node(),state=self.state)


'I have generated a presentation about the blind snake in the Bahamas.\n'

In [19]:
agent.deps.deep_search_results[0]

'# Blind Snakes of the Bahamas\n\nBlind snakes, belonging to the family Typhlopidae, are small, burrowing snakes with a worm-like appearance. Several species are found in the Bahamas, including the Bahamian slender blind snake (Cubatyphlops biminiensis) and the Brown Blind Snake (Typhlops lumbricalis).\n\n## Species\n\n*   **Bahamian Slender Blind Snake (Cubatyphlops biminiensis):** This species is endemic to the Bahamas. Its conservation status is listed as Least Concern.\n*   **Brown Blind Snake (Typhlops lumbricalis):** This species is found on most of the larger islands of the Bahamas, including the Abacos, Andros, the Berry Islands, Eleuthera, Cat Island, Long Island, and throughout the Exuma chain. On Bimini, they are commonly found in rotting logs and under rocks.\n\n## Characteristics\n\nBlind snakes are typically small and slender, resembling worms. They are adapted for burrowing, with smooth scales and reduced eyes.\n\n## Habitat and Distribution\n\nBahamian blind snakes inha

In [40]:
print(agent.deps.presentation_slides['page_1'])

# Blind Snakes of the Bahamas

## Overview

This presentation summarizes information about blind snakes found in the Bahamas.

---

## Species Overview

At least two native species of blind snakes are found in the Bahamas:

*   Bahamian Slender Blind Snake (*Cubatyphlops biminiensis*): Endemic to the Bahamas.
*   Brown Blind Snake (*Typhlops lumbricalis*): Found on most of the larger islands of the Bahamas.

---

## Bahamian Slender Blind Snake

*Cubatyphlops biminiensis*

*   Member of the Typhlopidae family.
*   Also referred to as the Bahamian Worm Snake.

---

## Brown Blind Snake

*Typhlops lumbricalis*

*   Small, thin snake.
*   Commonly found in rotting logs and under rocks on Bimini.

---

## Additional Considerations

*   The Brahminy Blind Snake (*Indotyphlops braminus*) has been introduced to Abaco Island in the Bahamas.

---

## References

*   [animalia.bio/bahamian-slender-blind-snake](https://animalia.bio/bahamian-slender-blind-snake)
*   [en.wikipedia.org/wiki/Bahamian

In [18]:
agent.memory.messages

[ModelRequest(parts=[SystemPromptPart(content='you are a research assistant, you are given a query, leverage what tool(s) to use,                          always show the output of the tools, except for the presentation agent', timestamp=datetime.datetime(2025, 4, 1, 15, 55, 52, 601180, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='do a deep research on the blind snake in the bahamas and generate a presentation about it', timestamp=datetime.datetime(2025, 4, 1, 15, 55, 52, 601187, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='deep_research_agent', args={'query': 'blind snake in the bahamas'}, tool_call_id='pyd_ai_1f16b0ddb435464dbb24d08c59c665ba', part_kind='tool-call')], model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 4, 1, 15, 55, 53, 741664, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='de